In [3]:
from facebook_page_scraper import Facebook_scraper
import os

page_group_or_name = "409901703293362" 
posts_count = 10
browser = "firefox"
proxy = "IP:PORT"
timeout = 600
fb_password = os.getenv('FACEBOOK_PASSWORD')
fb_email = os.getenv('FACEBOOK_EMAIL')
is_private = True
isGroup = True

scraper = Facebook_scraper(
  page_group_or_name, 
  posts_count, 
  browser, 
  proxy=proxy, 
  timeout=timeout, 
  isGroup=isGroup,
  username=fb_email,
  password=fb_password
)


In [ ]:
json_data = scraper.scrap_to_json()

2025-03-23 19:31:13,735 - facebook_page_scraper.driver_initialization - INFO - Using: IP:PORT
[WDM] - Driver [C:\Users\12488\.wdm\drivers\geckodriver\win64\v0.36.0\geckodriver.exe] found in cache


new layout loaded
all_posts length: 4
no post_url, skipping
no post_url, skipping
no post_url, skipping


In [ ]:
json_data

In [ ]:

import json
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
data = json.loads(json_data)

posts = []
for key, value in data.items():
    content = value.get("content", "").strip()
    posts.append(content)

sia = SentimentIntensityAnalyzer()

sentiment_results = []
positive_count = 0
negative_count = 0
neutral_count = 0

print("Individual Post Sentiment:\n")

for post in posts:
    scores = sia.polarity_scores(post)
    sentiment_results.append(scores)
    
    print(f"Post: {post}")
    print(f"Scores: {scores}\n")
    
    compound = scores['compound']
    if compound >= 0.05:
        positive_count += 1
    elif compound <= -0.05:
        negative_count += 1
    else:
        neutral_count += 1

n_posts = len(sentiment_results)
if n_posts > 0:
    avg_compound = sum(s['compound'] for s in sentiment_results) / n_posts
    avg_pos = sum(s['pos'] for s in sentiment_results) / n_posts
    avg_neg = sum(s['neg'] for s in sentiment_results) / n_posts
    avg_neu = sum(s['neu'] for s in sentiment_results) / n_posts

    print("---------------------------------------------------------")
    print(f"Total posts analyzed: {n_posts}")
    print(f"Positive posts: {positive_count}")
    print(f"Negative posts: {negative_count}")
    print(f"Neutral posts:  {neutral_count}")
    print("\nOverall Average Sentiment Scores (across all posts):")
    print(f"  Compound: {avg_compound:.3f}")
    print(f"  Positive: {avg_pos:.3f}")
    print(f"  Negative: {avg_neg:.3f}")
    print(f"  Neutral:  {avg_neu:.3f}")
    print("\nConclusion:")
    if avg_compound >= 0.05:
        print("Overall sentiment is generally **positive**.")
    elif avg_compound <= -0.05:
        print("Overall sentiment is generally **negative**.")
    else:
        print("Overall sentiment is **neutral or mixed**.")

else:
    print("No valid posts found for sentiment analysis.")